#  Import

In [ ]:
import os
import sys

import numpy as  np
from lwm_multi_model import multi_modal_lwm  # 클래스 import
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset as TorchDataset, DataLoader, Subset

from PIL import Image
import numpy as np

from deepverse import ParameterManager
from deepverse.scenario import ScenarioManager
from deepverse import Dataset

from deepverse.visualizers import ImageVisualizer, LidarVisualizer

# Settings

In [ ]:
# Scenes 2000
## Subcarriers 64

scenarios_name = "DT31"
config_path = f"scenarios/{scenarios_name}/param/config.m"
param_manager = ParameterManager(config_path)

params = param_manager.get_params()

param_manager.params["scenes"] =list(range(11000))
param_manager.params["comm"]["OFDM"]["selected_subcarriers"] = list(range(64))


# Generate a dataset

In [ ]:
dataset = Dataset(param_manager)

Generating camera dataset: ⏳ In progress
Generating camera dataset: ✅ Completed (0.00s)
Generating LiDAR dataset: ⏳ In progress
Generating LiDAR dataset: ✅ Completed (0.00s)
Generating mobility dataset: ⏳ In progress
Generating mobility dataset: ✅ Completed (0.00s)
Generating comm dataset: ⏳ In progress


Generating comm dataset: ✅ Completed (6.24s)
Generating radar dataset: ⏳ In progress


Generating radar dataset: ✅ Completed (214.41s)


# location dataset 
 지금 실험에서는 안쓰임

In [ ]:
# comm = dataset.comm_dataset
# location =  comm

# location = [
#     {
#         "bs_loc": d["bs_loc"],                      # (3,)
#         "ue_loc": np.asarray(d["ue_loc"]).squeeze() # (3,)  (원래 (1,3)이면 squeeze)
#     }
#     for row in comm.data      # row: [dict] 형태
#     for d in row              # d: dict
# ]

# print(ue_location)  #)

# communication  dataset

In [ ]:
# UE 정보
comm = dataset.comm_dataset
ch = comm.data[0][0]['ue'][0]
print(ch.coeffs.shape)  

(1, 16, 64)


# preprocessing

In [ ]:
def preprocess_channel_coeffs(coeffs_np: np.ndarray,
                              to_complex64: bool = True,
                              concat_dim:  int = -1,
                              device:  str = "cuda") -> torch.Tensor:
    """
    coeffs_np : 현재 복소형태 배열 e.g (1,16,64) 1=Bs, 16=Ue, 64=Subcarriers 
    """
    if not isinstance(coeffs_np, np.ndarray):
        raise TypeError(f"coeffs_np must be np.ndarray, got {type(coeffs_np)}")
    if not np.iscomplexobj(coeffs_np):
        raise TypeError(f"coeffs_np must be complex, got dtype={coeffs_np.dtype}")

    coeffs = torch.from_numpy(coeffs_np) 
    if to_complex64:
        coeffs = coeffs.to(torch.complex64)
    
    # real,imag concat
    H = torch.cat([coeffs.real, coeffs.imag], dim=concat_dim)
    H = H.to(device)
    return H

In [ ]:
# 사용예시
H = preprocess_channel_coeffs(ch.coeffs, to_complex64=True, concat_dim=-1)
print(H.shape)  # (1, 16, 128) 64 subcar

torch.Size([1, 16, 128])


### image dataset

In [ ]:
sensor = dataset.camera_dataset.sensors["unit1_cam1"]
path0 = sensor.files[0]
img = Image.open(path0).convert("RGB")
arr = np.array(img)

print("path:", path0)
print("PIL size (W,H):", img.size)
print("np shape:", arr.shape, "dtype:", arr.dtype)  # 보통 (H,W,3), uint8


path: scenarios/DT31/RGB_images/unit1_cam1/0.png
PIL size (W,H): (1920, 1080)
np shape: (1080, 1920, 3) dtype: uint8


In [ ]:
from PIL import Image
import numpy as np
import torch
import torch.nn.functional as F

IMG_SIZE = 224

def preprocess_img(path, img_size=IMG_SIZE, device="cuda"):
    # 1) load (H,W,3) uint8
    img = Image.open(path).convert("RGB")
    arr = np.array(img)

    # 2) numpy -> torch, (3,H,W), float32
    x = torch.from_numpy(arr).permute(2, 0, 1).contiguous().float()
    x = x / 255.0  # [0,1]

    # 3) add batch dim -> (1,3,H,W)
    x = x.unsqueeze(0)

    # 4) resize -> (1,3,224,224)
    x = F.interpolate(x, size=(img_size, img_size),
                      mode="bilinear", align_corners=False)

    # 5) normalize (ImageNet)
    mean = torch.tensor([0.485, 0.456, 0.406], dtype=x.dtype).view(1, 3, 1, 1)
    std  = torch.tensor([0.229, 0.224, 0.225], dtype=x.dtype).view(1, 3, 1, 1)
    x = (x - mean) / std

    # 6) move to device (GPU)
    x = x.to(device, non_blocking=True)

    return x  # (1,3,224,224) on device


In [ ]:
# 사용예시
cd = dataset.camera_dataset
sensor = cd.sensors['unit1_cam1']
path0 = sensor.files[0]
img = preprocess_img(path0, device="cuda")
print(img.shape, img.device)  # torch.Size([1,3,224,224]) cuda:0
print(path0)

torch.Size([1, 3, 224, 224]) cuda:0
scenarios/DT31/RGB_images/unit1_cam1/0.png


# Dataset 구현

In [ ]:
def get_coeffs_from_frame(frame, ue_idx=0):
    ue_obj = frame["ue"]

    # 케이스1) list/tuple이면 ue_idx로 선택
    if isinstance(ue_obj, (list, tuple)):
        ch_obj = ue_obj[ue_idx]
    else:
        # 케이스2) 단일 OFDMChannel이면 그대로 사용
        ch_obj = ue_obj

    # coeffs는 dict key가 아니라 attribute일 확률이 매우 큼
    if hasattr(ch_obj, "coeffs"):
        return ch_obj.coeffs

    # 혹시 dict라면 마지막 보험
    if isinstance(ch_obj, dict) and "coeffs" in ch_obj:
        return ch_obj["coeffs"]

    raise TypeError(f"Cannot get coeffs. ue type={type(ue_obj)}, ch type={type(ch_obj)}")


In [ ]:
def flatten_comm_frames(comm):
    frames = []
    for row in comm.data:
        for d in row:
            frames.append(d)
    return frames

class MultiModalNextStepDatasetGPU(TorchDataset):
    """
    B : Batch_size
    T : Time // past windwow length 각 샘플 시점 15개 + 1(예측)
    F : Feauture dimension per time step (각 시점에서 채널을 숫자 벡터로 펼친 길이) {64 subcarriers * 2 (real, imag) * num_tx * num_rx} 
    = 64*2*1*16=2048
    """
    def __init__(self, comm_frames, cam_files, ue_idx=0, past_len=16, device="cuda"):
        self.comm_frames = comm_frames
        self.cam_files = list(cam_files)
        self.ue_idx = ue_idx
        self.past_len = past_len
        self.device = device

        self.N = min(len(self.comm_frames), len(self.cam_files))
        self.valid_start = past_len - 1
        self.valid_end = self.N - 2  # need t+1
        if self.valid_end < self.valid_start:
            raise ValueError("Not enough samples for past_len+1.")

    def __len__(self):
        return self.valid_end - self.valid_start + 1

    def __getitem__(self, idx):
        t = self.valid_start + idx

        # image (GPU)
        img_path = self.cam_files[t]
        img = preprocess_img(img_path, device=self.device).squeeze(0)  # (3,224,224)

        # channel window (GPU)
        ch_list = []
        for k in range(t - self.past_len + 1, t + 1):
            coeffs_np = get_coeffs_from_frame(self.comm_frames[k], ue_idx=self.ue_idx)
            h = preprocess_channel_coeffs(coeffs_np, device=self.device)  # (1,16,128) or whatever
            h = h.reshape(-1)  # ✅ 모델 입력에 맞게 (F,)로 만들거나, 너 설계대로 유지
            ch_list.append(h)
        channel_past = torch.stack(ch_list, dim=0)  # (T, F)

        # target (GPU)
        coeffs_np_next = get_coeffs_from_frame(self.comm_frames[t + 1], ue_idx=self.ue_idx)
        target = preprocess_channel_coeffs(coeffs_np_next, device=self.device).reshape(-1)  # (F,)

        return channel_past, img, target


# DataLoader 구현

In [ ]:
comm_frames = flatten_comm_frames(dataset.comm_dataset)
sensor = dataset.camera_dataset.sensors["unit1_cam1"]

ds = MultiModalNextStepDatasetGPU(
    comm_frames=comm_frames,
    cam_files=sensor.files,
    ue_idx=0,
    past_len=16,
    device="cuda"
)

loader = DataLoader(
    ds,
    batch_size=8,
    shuffle=True,
    num_workers=0,     
    pin_memory=False   # ✅ 의미 없음 (이미 GPU)
)

ch, img, y = next(iter(loader))
print(ch.shape, img.shape, y.shape)
print(ch.device, img.device, y.device)


torch.Size([8, 16, 2048]) torch.Size([8, 3, 224, 224]) torch.Size([8, 2048])
cuda:0 cuda:0 cuda:0


# Fine-tuning
data shape 맞추기 위해

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from lwm_multi_model import multi_modal_lwm  # 너가 올린 backbone

class FinetuneChannelPredictor(nn.Module):
    """
    직접 구현한 lwm_multi_model(channel + image)에 맞는 파인튜닝 모델
    Input:
      ch:  (B, T, F_in)  e.g., (B,16,2048) 16: past Length, 2048: feature dim
      img: (B, 3, 224, 224)
    Output:
      yhat: (B, F_out)  e.g., (B,2048)
    """
    def __init__(
        self,
        backbone: nn.Module,
        F_in: int,
        F_out: int,
        pool: str = "last",          # "last" or "mean"
        freeze_image: bool = False,
        freeze_backbone: bool = False,
        element_length: int = 16,    # 채널 벡터 차원 (backbone 기대값)
        d_model: int = 64            # backbone 내부 feature dim
    ):
        super().__init__()
        self.backbone = backbone
        self.pool = pool

        # backbone이 기대하는 channel feature dim = ELEMENT_LENGTH
        # (backbone 내부 Channel_Embedding: Linear(ELEMENT_LENGTH -> D_MODEL))
        if element_length is None:
            element_length = backbone.channel_embedding.element_length
        if d_model is None:
            d_model = backbone.channel_embedding.d_model

        # 입력 차원 정렬: F_in -> ELEMENT_LENGTH
        self.in_proj = nn.Sequential(
            nn.Linear(F_in, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, element_length)
        )

        # 출력 head: D_MODEL -> F_out
        self.head = nn.Linear(d_model, F_out)

        if freeze_image:
            for p in self.backbone.image_embedding.parameters():
                p.requires_grad = False

        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False
            # 그래도 projection/head는 학습되게 다시 켜기
            for p in self.in_proj.parameters():
                p.requires_grad = True
            for p in self.head.parameters():
                p.requires_grad = True

    def forward(self, ch, img):
        # ch: (B,T,F_in) -> (B,T,ELEMENT_LENGTH)
        ch = self.in_proj(ch)

        # backbone: (B,T,D_MODEL)
        tokens = self.backbone(ch, img)

        # pooling -> (B,D_MODEL)
        if self.pool == "last":
            z = tokens[:, -1, :]
        elif self.pool == "mean":
            z = tokens.mean(dim=1)
        else:
            raise ValueError(f"Unknown pool={self.pool}")

        # head -> (B,F_out)
        yhat = self.head(z)
        return yhat


## NMSE(dB)

In [ ]:
@torch.no_grad()
def nmse_db(yhat: torch.Tensor, y: torch.Tensor, eps: float = 1e-12) -> torch.Tensor:
    # yhat, y: (B,F)
    num = torch.sum((yhat - y) ** 2, dim=1)
    den = torch.sum(y ** 2, dim=1).clamp_min(eps)
    nmse = num / den
    return 10.0 * torch.log10(nmse.clamp_min(eps)).mean()


# Train/Val split

In [ ]:
n = len(ds)
n_train = int(0.75 * n)
train_idx = list(range(0, n_train))
val_idx = list(range(n_train, n))

train_ds = Subset(ds, train_idx)
val_ds   = Subset(ds, val_idx)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=0)

In [ ]:
comm_frames = flatten_comm_frames(dataset.comm_dataset)
cam_files = list(dataset.camera_dataset.sensors["unit1_cam1"].files)

print("len(comm_frames):", len(comm_frames))
print("len(cam_files):", len(cam_files))
print("first comm frame keys:", list(comm_frames[0].keys()))
print("first cam file:", cam_files[0])


len(comm_frames): 100
len(cam_files): 7012
first comm frame keys: ['bs_loc', 'ue', 'ue_loc', 'bs']
first cam file: scenarios/DT31/RGB_images/unit1_cam1/0.png


In [ ]:
len(val_loader)

1

# Model generate and also check

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# 배치 하나로 F_in/F_out 자동 확정
ch, img, y = next(iter(train_loader))
F_in  = ch.shape[-1]
F_out = y.shape[-1]
print("Detected:", "F_in=", F_in, "F_out=", F_out)
print("Batch devices:", ch.device, img.device, y.device)

# backbone + finetune model
backbone = multi_modal_lwm().to(device)

model = FinetuneChannelPredictor(
    backbone=backbone,
    F_in=F_in,
    F_out=F_out,
    pool="last",            # "mean"으로 바꿔도 됨
    freeze_image=False,     # 원하면 True (이미지 인코더 고정)
    freeze_backbone=False,  # 원하면 True (proj/head만 학습)
    element_length=128,
    d_model=256
).to(device)

# sanity forward
model.eval()
with torch.no_grad():
    # ds가 이미 cuda 텐서 반환이면 아래 .to(device) 생략 가능
    yhat = model(ch.to(device), img.to(device))
print("yhat:", yhat.shape, "y:", y.shape)



device: cuda
Detected: F_in= 2048 F_out= 2048
Batch devices: cuda:0 cuda:0 cuda:0
yhat: torch.Size([32, 2048]) y: torch.Size([32, 2048])


# Train/ Eval 함수 (AMP + grad clip)

In [ ]:
def train_one_epoch(model, loader, optimizer, device, use_amp=True, grad_clip=1.0):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    total_loss = 0.0
    total_nmse = 0.0
    n = 0

    for ch, img, y in loader:
        # Dataset이 이미 cuda 텐서를 반환하더라도 안전하게 유지
        ch = ch.to(device, non_blocking=True)
        img = img.to(device, non_blocking=True)
        y  = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=use_amp):
            yhat = model(ch, img)
            loss = F.mse_loss(yhat, y)

        scaler.scale(loss).backward()

        if grad_clip is not None and grad_clip > 0:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        total_nmse += nmse_db(yhat.detach(), y).item()
        n += 1

    return total_loss / max(n, 1), total_nmse / max(n, 1)


@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()

    total_loss = 0.0
    total_nmse = 0.0
    n = 0

    for ch, img, y in loader:
        ch = ch.to(device, non_blocking=True)
        img = img.to(device, non_blocking=True)
        y  = y.to(device, non_blocking=True)

        yhat = model(ch, img)
        loss = F.mse_loss(yhat, y)

        total_loss += loss.item()
        total_nmse += nmse_db(yhat, y).item()
        n += 1

    return total_loss / max(n, 1), total_nmse / max(n, 1)


#  Optiimizer  / Scheduler 설정

In [ ]:
# requires_grad=True인 파라미터만 학습
trainable_params = [p for p in model.parameters() if p.requires_grad]
print("trainable params:", sum(p.numel() for p in trainable_params))

optimizer = torch.optim.AdamW(trainable_params, lr=1e-4, weight_decay=1e-4)

# (선택) cosine scheduler
epochs = 1000
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)


trainable params: 20839648


# 학습 루프 + checkpoint 저장

In [ ]:
best_val = float("inf")

for epoch in range(1, epochs + 1):
    t0 = time.time()

    tr_loss, tr_nmse = train_one_epoch(model, train_loader, optimizer, device=device, use_amp=True, grad_clip=1.0)
    va_loss, va_nmse = evaluate(model, val_loader, device=device)

    scheduler.step()

    dt = time.time() - t0
    print(
        f"[{epoch:02d}/{epochs}] "
        f"train loss={tr_loss:.6f}, nmse(dB)={tr_nmse:.2f} | "
        f"val loss={va_loss:.6f}, nmse(dB)={va_nmse:.2f} | "
        f"{dt:.1f}s"
    )

    if va_loss < best_val:
        best_val = va_loss
        torch.save(
            {
                "epoch": epoch,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "F_in": F_in,
                "F_out": F_out,
            },
            "best_finetune.pt"
        )
        print("  ↳ saved best_finetune.pt")


/tmp/ipykernel_39231/900206524.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
/tmp/ipykernel_39231/900206524.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


[01/1000] train loss=0.296778, nmse(dB)=122.64 | val loss=0.182845, nmse(dB)=127.05 | 8.8s
  ↳ saved best_finetune.pt
[02/1000] train loss=0.189053, nmse(dB)=120.72 | val loss=0.131645, nmse(dB)=125.63 | 4.8s
  ↳ saved best_finetune.pt
[03/1000] train loss=0.142960, nmse(dB)=119.50 | val loss=0.098874, nmse(dB)=124.38 | 4.7s
  ↳ saved best_finetune.pt
[04/1000] train loss=0.113830, nmse(dB)=118.53 | val loss=0.075701, nmse(dB)=123.22 | 4.9s
  ↳ saved best_finetune.pt
[05/1000] train loss=0.092483, nmse(dB)=117.63 | val loss=0.057654, nmse(dB)=122.04 | 4.8s
  ↳ saved best_finetune.pt
[06/1000] train loss=0.074127, nmse(dB)=116.67 | val loss=0.043040, nmse(dB)=120.77 | 4.8s
  ↳ saved best_finetune.pt
[07/1000] train loss=0.062110, nmse(dB)=115.90 | val loss=0.031598, nmse(dB)=119.43 | 7.6s
  ↳ saved best_finetune.pt
[08/1000] train loss=0.051399, nmse(dB)=115.06 | val loss=0.023179, nmse(dB)=118.08 | 4.7s
  ↳ saved best_finetune.pt
[09/1000] train loss=0.044904, nmse(dB)=114.49 | val los

KeyboardInterrupt: 

In [ ]:
ch, img, y = next(iter(train_loader))
print("y abs mean:", y.abs().mean().item())
print("y abs max :", y.abs().max().item())
print("y power   :", (y**2).mean().item())

with torch.no_grad():
    yhat = model(ch.to(device), img.to(device))
print("yhat abs mean:", yhat.abs().mean().item())
print("yhat abs max :", yhat.abs().max().item())
print("yhat power   :", (yhat**2).mean().item())


y abs mean: 3.6235508105164627e-07
y abs max : 1.0917989357039914e-06
y power   : 1.815636670957771e-13
yhat abs mean: 0.14525052905082703
yhat abs max : 0.8965445756912231
yhat power   : 0.03346424549818039


# 데이터 입력 및 형태

In [ ]:
print("=== dataset sizes ===")
print("N(comm_frames):", len(comm_frames))
print("N(cam_files)  :", len(cam_files))
print("N(min)        :", min(len(comm_frames), len(cam_files)))
print("past_len      :", ds.past_len)
print("len(ds)       :", len(ds))
print("len(train_ds) :", len(train_ds))
print("len(val_ds)   :", len(val_ds))
print("len(train_loader):", len(train_loader))
print("len(val_loader)  :", len(val_loader))

print("\n=== one batch shapes ===")
ch, img, y = next(iter(train_loader))
print("ch :", tuple(ch.shape), " -> (B,T,F_in)")
print("img:", tuple(img.shape), " -> (B,3,224,224)")
print("y  :", tuple(y.shape), " -> (B,F_out)")
with torch.no_grad():
    yhat = model(ch.to(device), img.to(device))
print("yhat:", tuple(yhat.shape), " -> (B,F_out)")
print("this forward predicted vectors:", yhat.shape[0], "(=B)")
print("each vector predicts elements:", yhat.shape[1], "(=F_out)")


=== dataset sizes ===
N(comm_frames): 100
N(cam_files)  : 7012
N(min)        : 100
past_len      : 16
len(ds)       : 84
len(train_ds) : 63
len(val_ds)   : 21
len(train_loader): 2
len(val_loader)  : 1

=== one batch shapes ===
ch : (32, 16, 2048)  -> (B,T,F_in)
img: (32, 3, 224, 224)  -> (B,3,224,224)
y  : (32, 2048)  -> (B,F_out)
yhat: (32, 2048)  -> (B,F_out)
this forward predicted vectors: 32 (=B)
each vector predicts elements: 2048 (=F_out)
